### Hypothesis

1. Which country has more attacks?
2. Attacks increase fatality depending on Year and Age?
3. Show attacks evolution by year (less/more sharks, less/mnore agressive)
4. Which type of acitvity is done when attack? Any activity more dangerous than others?


### Needed actions
1. Setup and Libraries
2. Get all info from dataset:
    - Read Dataset in a new frame
    - Describe
    - Shape
    - Columns 
    - Types
3. Fix columns
    - Strip and upper names
4. Null values and dropping
    - Check amount of NaN
    - Drop not needed colums
    - Drop rows with all Nan
5. Fixing content needed columns
    - Replace Nan with '' or 0
    - Case Number to date
    - COUNTRY
    - Year
    - Age
    - Type
    - Activity
6. Visualize hypothesis using charts


**1. Setup and Libraries:**

In [36]:
#Libraries required

import pandas as pd
import numpy as np 
import re as r


#setup visualize
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

#Read data from external source
df_sharks = pd.read_csv("../data/attacks.csv", sep=",", encoding="Latin-1")
df_sharks.head()



,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and paddle damaged",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.18-McNeely.pdf,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.09-Denges.pdf,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.08-Arrawarra.pdf,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.04-Ramos.pdf,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [2]:
# Quick look and df features

print(f"Shape size:{df_sharks.shape}\n")
print(f"Columns: {df_sharks.columns}\n")
print(f"Types: {df_sharks.dtypes}")



Shape size:(25723, 24)

Columns: Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Types: Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object


In [3]:
#Fix columns names: strip and uppercase

def fix_cols(df):
    clean_lst = []
    for c in df.columns:
        c = c.strip().upper()
        clean_lst.append(c)
    df.columns = clean_lst
    return df.columns

def change_col(df,old,new):
    df.rename(columns={old:str.upper(new)},inplace = True)

fix_cols(df_sharks)
change_col(df_sharks,"FATAL (Y/N)","FATAL")
change_col(df_sharks,"INVESTIGATOR OR SOURCE","SOURCE")


In [4]:
print(f"{df_sharks['HREF'].isin(df_sharks['HREF FORMULA']).value_counts()}") # pretty similar columns
print(f"{df_sharks['CASE NUMBER'].isin(df_sharks['CASE NUMBER.1']).value_counts()}") # pretty similar columns
print(f"{df_sharks['CASE NUMBER'].isin(df_sharks['CASE NUMBER.2']).value_counts()}") # pretty similar columns
print(f"{df_sharks['CASE NUMBER.1'].isin(df_sharks['CASE NUMBER.2']).value_counts()}") # pretty similar columns

True     25666
False       57
Name: HREF, dtype: int64
True     23299
False     2424
Name: CASE NUMBER, dtype: int64
True     23319
False     2404
Name: CASE NUMBER, dtype: int64
True     25704
False       19
Name: CASE NUMBER.1, dtype: int64


In [5]:

#Selecting columns to work: dropping

df_sharks.drop(['UNNAMED: 22','UNNAMED: 23','HREF FORMULA','CASE NUMBER.1','CASE NUMBER.2','PDF','ORIGINAL ORDER','SOURCE','AREA','LOCATION'], axis=1,inplace=True) # These columns don't have interestin data for our goal


In [6]:
# Null values actions

# % of null values in every column
for col in df_sharks.columns:
    v_missed = np.mean(df_sharks[col].isna())
    print(f"{col} - {round(v_missed*100)}%")



CASE NUMBER - 66%
DATE - 76%
YEAR - 76%
TYPE - 76%
COUNTRY - 76%
ACTIVITY - 78%
NAME - 76%
SEX - 78%
AGE - 87%
INJURY - 76%
FATAL - 78%
TIME - 89%
SPECIES - 87%
HREF - 76%


In [7]:
df_sharks["CASE NUMBER"].replace('0',np.nan,inplace=True)
df_sharks["CASE NUMBER"].replace('xx',np.nan,inplace=True)

In [8]:
#Clean empty rows
df_sharks.dropna(how="all",inplace=True)
df_sharks.fillna('Unknown')

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,ACTIVITY,NAME,SEX,AGE,INJURY,FATAL,TIME,SPECIES,HREF
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and paddle damaged",N,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.18-McNeely.pdf
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.09-Denges.pdf
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,Surfing,male,M,Unknown,Minor injury to lower leg,N,Unknown,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.08-Arrawarra.pdf
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Free diving,Gustavo Ramos,M,Unknown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Unknown,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.04-Ramos.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Diving,male,M,Unknown,FATAL,Y,Unknown,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0005-RoebuckBay.pdf
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Pearl diving,Ahmun,M,Unknown,FATAL,Y,Unknown,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0004-Ahmun.pdf
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,Swimming,Coast Guard personnel,M,Unknown,FATAL,Y,Unknown,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0003-Ocracoke_1900-1905.pdf
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,Unknown,Jules Patterson,M,Unknown,FATAL,Y,Unknown,Unknown,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0002-JulesPatterson.pdf


In [9]:
df_sharks.shape

(6302, 14)

### Fixing content inside columns

### CASE NUMBER fixing

In [10]:
# CASE NUMBER
'''
- Review content
- Check consistency
'''

for col in df_sharks.columns:
    v_missed = np.mean(df_sharks[col].isna())
    print(f"{col} - {round(v_missed*100)}%")

CASE NUMBER - 0%
DATE - 0%
YEAR - 0%
TYPE - 0%
COUNTRY - 1%
ACTIVITY - 9%
NAME - 3%
SEX - 9%
AGE - 45%
INJURY - 0%
FATAL - 9%
TIME - 53%
SPECIES - 45%
HREF - 0%


In [11]:

pd.set_option('display.max_rows', 100)

df_sharks["CASE NUMBER"].fillna('0',inplace=True)
df_sharks["CASE NUMBER"] = df_sharks["CASE NUMBER"].str[:10] 
df_sharks['CASE NUMBER'] = df_sharks['CASE NUMBER'].str.replace('-', '.', case=False, regex=True)
df_sharks['CASE NUMBER'] = df_sharks['CASE NUMBER'].str.replace(',', '.', case=False, regex=True)
df_sharks.reindex()

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,ACTIVITY,NAME,SEX,AGE,INJURY,FATAL,TIME,SPECIES,HREF
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and paddle damaged",N,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.18-McNeely.pdf
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.09-Denges.pdf
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.08-Arrawarra.pdf
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.04-Ramos.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Diving,male,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0005-RoebuckBay.pdf
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0004-Ahmun.pdf
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0003-Ocracoke_1900-1905.pdf
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0002-JulesPatterson.pdf


In [12]:
df_sharks.shape

(6302, 14)

In [13]:
df_sharks['CASE NUMBER'].isnull().sum()


0

In [14]:

for x in df_sharks.index:  
    if len(df_sharks["CASE NUMBER"].values[x]) == 10:

        v_year = r.search('(\d{4})\.(\d{2})\.(\d{2})',df_sharks.loc[x, "CASE NUMBER"]).group(1)
        v_month = r.search('(\d{4})\.(\d{2})\.(\d{2})',df_sharks.loc[x, "CASE NUMBER"]).group(2)
        v_day = r.search('(\d{4})\.(\d{2})\.(\d{2})',df_sharks.loc[x, "CASE NUMBER"]).group(3)
        df_sharks.loc[x, "YEAR_"] = v_year
        df_sharks.loc[x, "MONTH_"] = v_month
        df_sharks.loc[x, "DAY_"] = v_day
        df_sharks.loc[x, "YEAR"] = v_year
        df_sharks.loc[x, "DATE"] = v_day+"/"+v_month+"/"+v_year
    else:
        df_sharks.loc[x, "YEAR_"] = 0
        df_sharks.loc[x, "MONTH_"] = 0
        df_sharks.loc[x, "DAY_"] = 0
        df_sharks.loc[x, "YEAR"] = 0
        df_sharks.loc[x, "DATE"] = 0


In [15]:
pd.set_option('display.max_rows', None)

df_sharks.drop(df_sharks[df_sharks["YEAR"] == 0].index, inplace=True)
df_sharks.drop(df_sharks[df_sharks["YEAR"] == '0077'].index, inplace=True)
df_sharks.drop(df_sharks[df_sharks["YEAR"] == '0500'].index, inplace=True)
df_sharks.drop(df_sharks[df_sharks["YEAR"] == '0005'].index, inplace=True)


In [16]:
df_sharks["YEAR"]=df_sharks["YEAR"].astype(int)

### ACTIVITY column


In [17]:
df_sharks["ACTIVITY"].value_counts()


Surfing                                                                                                                                                                                                                                                           970
Swimming                                                                                                                                                                                                                                                          854
Fishing                                                                                                                                                                                                                                                           422
Spearfishing                                                                                                                                                                                                          

In [18]:
df_sharks["ACTIVITY"].fillna('Unknown',inplace=True)
df_sharks["ACTIVITY"].isna().sum()

0

In [19]:
df_sharks["ACTIVITY"].value_counts()


Surfing                                                                                                                                                                                                                                                           970
Swimming                                                                                                                                                                                                                                                          854
Unknown                                                                                                                                                                                                                                                           527
Fishing                                                                                                                                                                                                               

In [20]:
def fix_activity(string):
        string = str(string).lower().strip()
        if string != string:
            return np.nan
        elif "swimming" in string or "bathing" in string or "floating" in string or "splashing" in string or "jumped into the water" in string or "playing" in string:
            return "swimming"
        elif "diving" in string or "snorkel" in string:
            return "diving"
        elif "fish" in string:
            return "fishing"
        elif "surf" in string or "body boarding" in string or "body-boarding" in string or "boogie boarding" in string or "paddleskiing" in string:
            return "surf"
        elif "standing" in string or "paddling" in string or "paddl" in string:
            return "paddling"
        elif "kayaking" in string or "ship" in string or "sail" in string or "boat" in string or "canoeing" in string or "board" in string or "rowing" in string or "fell" in string:
            return "boating"
        elif "disaster" in string:
            return "sea disaster"
        elif "wading" in string or "walking" in string or "treading water" in string or "feed" in string:
            return "walking"
        else:
            return "others"
    



In [21]:
for i in df_sharks.index:
        v_string = df_sharks.loc[i, "ACTIVITY"]
        df_sharks.loc[i, "ACTIVITY"] = fix_activity(v_string)
        

In [22]:
df_sharks["ACTIVITY"].value_counts()

swimming        1367
surf            1304
fishing         1129
others           962
diving           636
boating          373
walking          221
paddling         151
sea disaster      29
Name: ACTIVITY, dtype: int64

### COUNTRY column fixing

In [23]:
#Check column

pd.set_option('display.max_rows', None)

value_counts = df_sharks['COUNTRY'].value_counts()
sorted_counts = value_counts.sort_index()
print(sorted_counts)



 PHILIPPINES                                1
 TONGA                                      3
ADMIRALTY ISLANDS                           1
AFRICA                                      1
ALGERIA                                     1
AMERICAN SAMOA                              3
ANDAMAN / NICOBAR ISLANDAS                  1
ANDAMAN ISLANDS                             1
ANGOLA                                      1
ANTIGUA                                     2
ARGENTINA                                   1
ARUBA                                       1
ATLANTIC OCEAN                             17
AUSTRALIA                                1321
AZORES                                      5
BAHAMAS                                   106
BANGLADESH                                  1
BARBADOS                                    6
BAY OF BENGAL                               1
BELIZE                                      2
BERMUDA                                    14
BRAZIL                            

In [24]:
#Cleaning COUNTRY column

df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.strip()
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('?', '',regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*UAE.*', 'UNITED ARAB EMIRATES',case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('TOBAGO', 'TRINIDAD & TOBAGO',case=False,regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*VANUATU.*', 'SOLOMON ISLANDS' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*RED SEA.*', 'RED SEA' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*REUNION.*', 'REUNION' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*ITALY.*', 'ITALY' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*IRAN.*', 'IRAN' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*EGYPT.*', 'EGYPT' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*CEYLON.*', 'CEYLON' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*ANDAMAN.*', 'ANDAMAN ISLANDS' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*EQUATORIAL GUINEA.*', 'EQUATORIAL GUINEA' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*MALDIVE ISLANDS.*', 'MALDIVES' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('.*ST HELENA.*', 'ST HELENA' ,case=False, regex=True)
df_sharks['COUNTRY'] = df_sharks['COUNTRY'].str.replace('Fiji', 'FIJI',regex=True)
df_sharks.drop(df_sharks[(df_sharks['COUNTRY']=='AFRICA')].index, inplace=True)
df_sharks.drop(df_sharks[(df_sharks['COUNTRY']=='Coast of AFRICA')].index, inplace=True)
df_sharks.drop(df_sharks[(df_sharks['COUNTRY']=='Between PORTUGAL & INDIA')].index, inplace=True)



In [25]:
value_counts = df_sharks['COUNTRY'].value_counts()
sorted_counts = value_counts.sort_index()
print(sorted_counts)

ADMIRALTY ISLANDS                    1
ALGERIA                              1
AMERICAN SAMOA                       3
ANDAMAN ISLANDS                      2
ANGOLA                               1
ANTIGUA                              2
ARGENTINA                            1
ARUBA                                1
ATLANTIC OCEAN                      17
AUSTRALIA                         1321
AZORES                               5
BAHAMAS                            106
BANGLADESH                           1
BARBADOS                             6
BAY OF BENGAL                        1
BELIZE                               2
BERMUDA                             14
BRAZIL                             112
BRITISH ISLES                        1
BRITISH NEW GUINEA                   1
BRITISH VIRGIN ISLANDS               1
BRITISH WEST INDIES                  1
BURMA                                2
CANADA                              11
CAPE VERDE                           3
CARIBBEAN SEA            

### SEX column fixing

In [26]:
df_sharks["SEX"] = df_sharks["SEX"].str.strip()

value_counts = df_sharks['SEX'].value_counts()
sorted_counts = value_counts.sort_index()
print(sorted_counts)
df_sharks['SEX'] = df_sharks['SEX'].str.replace('.*lli.*', 'M',regex=True)
df_sharks['SEX'] = df_sharks['SEX'].str.replace('.*N.*', 'M',regex=True)
df_sharks.drop(df_sharks[(df_sharks['SEX']=='.')].index, inplace=True)



.         1
F       624
M      4984
N         2
lli       1
Name: SEX, dtype: int64


In [27]:
value_counts = df_sharks['SEX'].value_counts()
sorted_counts = value_counts.sort_index()
print(sorted_counts)

F     624
M    4987
Name: SEX, dtype: int64


### AGE column fixing



In [28]:
df_sharks["AGE"].value_counts()
df_sharks["AGE"].isna().sum()

2710

In [29]:
df_sharks['AGE'] = pd.to_numeric(df_sharks["AGE"].astype(str).str.replace(',',''), errors='coerce').fillna(0).astype(int)

In [30]:
df_sharks["AGE"].value_counts()

0     2811
17     154
18     150
19     141
20     141
15     137
16     136
21     118
22     117
25     107
24     106
14     101
13      94
26      83
28      81
23      80
29      78
27      77
30      77
12      73
35      69
32      69
10      56
40      56
31      52
34      50
38      48
33      44
43      44
36      42
37      42
41      38
39      38
42      38
11      37
52      35
9       35
45      33
50      32
47      30
44      29
49      28
48      28
8       28
46      27
55      23
7       22
51      20
54      17
57      16
60      16
58      15
61      13
53      13
6       13
59      11
56      10
69      10
63       9
62       8
5        7
68       7
70       6
65       5
3        5
64       4
66       4
77       3
71       3
73       3
1        2
75       2
78       2
74       2
86       1
84       1
87       1
67       1
82       1
81       1
72       1
Name: AGE, dtype: int64

### FATAL column cleaning

In [31]:
df_sharks["FATAL"].value_counts()

N          4226
Y          1323
UNKNOWN      70
 N            7
M             1
2017          1
N             1
y             1
Name: FATAL, dtype: int64

In [32]:
df_sharks['FATAL'] = df_sharks['FATAL'].str.strip()
df_sharks['FATAL'] = df_sharks['FATAL'].str.upper()
df_sharks['FATAL'] = df_sharks['FATAL'].replace('M','N')
df_sharks['FATAL'] = df_sharks['FATAL'].replace('2017','N')




In [33]:
df_sharks["FATAL"].value_counts()

N          4236
Y          1324
UNKNOWN      70
Name: FATAL, dtype: int64

In [35]:
df_sharks.to_csv("../data/attacks_chart.csv")
